In [111]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt
import copy
from mpl_toolkits.mplot3d import axes3d, Axes3D
from matplotlib import cm
import math
import pandas as pd
import random

# Algoritmos de Optimización


# Ant Colony Optimization

### ACO

In [112]:
def ncamino(ciudades, costos, feromonas):
    
    aux = copy.copy(ciudades)
    ruta = []
    pos = random.choice(aux)
    poso = pos
    ruta.append(pos)
    aux.remove(pos)
    suma = 0
    
    #Creamos un camino para una hormiga
    while len(ruta) < len(ciudades):
        
        prob = list()
                
        if len(aux) == 1:
            ruta.append(aux[0])
        else:
            #Sumamos los puntos vecinos
            for i in aux:
                suma += (feromonas[pos][i] * (1/costos[pos][i]))
                
            #Obtenemos probabilidad de cada punto vecino
            for i in aux:
                prob.append((feromonas[pos][i] * (1/costos[pos][i]))/suma)  
            
            #Obtenemos suma de cada punto y lo ordenamos
            union = zip(aux, prob)
            cdsort = list(union)
            cdsort = sorted(cdsort, key = lambda x: x[1]) 
            cdsort, prob = zip(*cdsort)
            sumas = np.cumsum(prob)
            
            #Seleccionamos un punto con un numero aleatorio
            sel= random.uniform(0,sumas[-1])
            c = True
            i = 0
            while c:
                if sel < sumas[i]:
                    pos = i
                    c = False
                i += 1
            
            #Agregamos el punto como parte del camino
            pos = cdsort[pos]
            ruta.append(pos)
            aux.remove(pos)
            
    ruta.append(poso)

    #Calculamos tau
    tau = 0    
    for i in range(len(ruta)-1):
        tau += costos[ruta[i]][ruta[i+1]]

    return ruta, tau

In [113]:
def ACO (costos, vap, ants, imax):
    
    #Inicializacion    
    col, fil= costos.shape
    feromonas = np.ones_like(costos).astype(float)        
    np.fill_diagonal(feromonas,0)
    ciudades = list(range(len(costos)))
    iteracion = 0    
    while iteracion <= imax:
        iteracion += 1        
        caminos = np.zeros((len(costos), (len(costos)+1))).astype(float)
        tau = np.zeros((len(costos))).astype(float)         
        for i in range(ants):
            #Las hormigas buscan un camino
            caminos[i], tau[i] = ncamino(
                                                    copy.copy(ciudades), 
                                                    copy.copy(costos), 
                                                    copy.copy(feromonas)
                                                    )        
        #Actualizamos la matriz de feromonas
        for i in range(caminos.shape[0]-1):
            for j in range(caminos.shape[1]-2):
                aux = feromonas[int(caminos[i][j])][int(caminos[i][j+1])]
                #Calculamos el nuevo valor de la feromona
                feromonas[int(caminos[i][j])][int(caminos[i][j+1])] = vap*aux + (1/tau[i])  
    return caminos

In [114]:
lDist = [[0,20,42,35],
        [20,0,30,34],
        [42,30,0,12],
        [35,34,12,0]]

Dist= np.array(lDist)
        
caminos = ACO(Dist, 0.5, 4, 50)

print('Finales:')
print(caminos)

Finales:
[[0. 1. 3. 2. 0.]
 [3. 2. 1. 0. 3.]
 [3. 2. 1. 0. 3.]
 [0. 1. 3. 2. 0.]]
